# MNIST 손글씨 - CNN v1
- 2 Conv2D layers
- 1 Fully Connected Network layer
- 1 Output layer

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

- 데이터 전처리

In [2]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# X data: 0~1 사이의 값, 3차원 --> 4차원
X_train = X_train.reshape(-1,28,28,1) / 255.
X_test = X_test.reshape(-1,28,28,1) / 255.

In [4]:
# One-hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

- 모델 정의/학습/평가

In [14]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),       # mask의 갯수, 크기 
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 512)               18874880  
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 18,898,826
Trainable params: 18,898,826
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model_path = 'best-mnist-cnn-1.h5'
checkpoint = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
early_stop = EarlyStopping(patience=10)

In [9]:
hist = model.fit(
    X_train, Y_train, validation_split=0.2, epochs=100, batch_size=200,
    callbacks=[checkpoint, early_stop], verbose=0
)

Epoch 1/100
238/240 [============================>.] - ETA: 0s - loss: 0.1529 - accuracy: 0.9529
Epoch 00001: val_loss improved from inf to 0.05638, saving model to best-mnist-cnn-1.h5
240/240 [==============================] - 21s 21ms/step - loss: 0.1521 - accuracy: 0.9531 - val_loss: 0.0564 - val_accuracy: 0.9833
Epoch 2/100
238/240 [============================>.] - ETA: 0s - loss: 0.0356 - accuracy: 0.9890
Epoch 00002: val_loss improved from 0.05638 to 0.05404, saving model to best-mnist-cnn-1.h5
240/240 [==============================] - 5s 20ms/step - loss: 0.0354 - accuracy: 0.9890 - val_loss: 0.0540 - val_accuracy: 0.9852
Epoch 3/100
240/240 [==============================] - ETA: 0s - loss: 0.0178 - accuracy: 0.9945
Epoch 00003: val_loss improved from 0.05404 to 0.04526, saving model to best-mnist-cnn-1.h5
240/240 [==============================] - 5s 20ms/step - loss: 0.0178 - accuracy: 0.9945 - val_loss: 0.0453 - val_accuracy: 0.9874
Epoch 4/100
238/240 [===================

In [15]:
best_model = load_model(model_path)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0413 - accuracy: 0.9871


[0.04130115360021591, 0.9871000051498413]